<a href="https://colab.research.google.com/github/NayeonKeum/ToyProjects/blob/master/cafe/income_subway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
import pandas as pd
import numpy as np

data=pd.read_csv("/content/drive/My Drive/Colab Notebooks/data_2020cafe/data_income_inheo.csv")

In [ ]:
data.head()

,Unnamed: 0,자치구명,행정동명,년도,인허가일자,폐업일자,영업상태코드,영업상태명,사업장명,X,Y,시설총규모,규모분기,총층수1,업소위치분기,영업기간,당월_매출_금액,당월_매출_건수,점포수,cluster_id,역이름,SX,SY,호선,distance,d_code
0,8,종로구,종로1.2.3.4가동,1990,1990-04-24,2014-11-24,2,폐업,명 다방,126.986202,37.572381,45.15,5,1.0,1,8980,385169402.0,28637.0,13.0,205.0,종각역,126.983134,37.570222,01호선,0.36159,2
1,96,종로구,종로1.2.3.4가동,1992,1992-11-27,2020-12-31,1,영업,세운다방,126.993508,37.569335,14.20,1,1.0,1,10261,152473261.0,43013.0,2.0,206.0,종로3가역,126.992144,37.570421,03호선,0.17035,1
2,153,종로구,종로1.2.3.4가동,1993,1993-09-08,2020-12-31,1,영업,그린파워,126.990956,37.569367,6.00,1,1.0,1,9976,385169402.0,28637.0,13.0,206.0,종로3가역,126.992144,37.570421,03호선,0.15715,1
3,165,종로구,종로1.2.3.4가동,1993,1993-11-01,2020-12-31,1,영업,영풍개발(주),126.979242,37.569580,106.12,9,5.0,1,9922,83213521.0,13240.0,6.0,18.0,광화문역,126.976410,37.571644,05호선,0.33909,2
4,168,종로구,종로1.2.3.4가동,1993,1993-11-05,2020-12-31,1,영업,향원휴게실,126.994251,37.569127,11.20,1,1.0,1,9918,206368601.0,17347.0,14.0,206.0,종로3가역,126.992144,37.570421,03호선,0.23488,1


In [ ]:
feat_drop=['개방자치단체코드','관리번호','업소일련번호', '폐업구분','상권_코드','총층수','업소위치']
data=data.drop(feat_drop, axis=1)

KeyError: ignored

In [ ]:
data.info()

In [ ]:
#년도 빈거 2019으로!
data["년도"]=data["년도"].fillna(2018)
data["총층수1"]=data["총층수1"].fillna(1)

#좌표를 채우기 평균으로 채우기
data['X'].fillna(data.groupby("행정동명")['X'].transform("mean"), inplace=True)
data['Y'].fillna(data.groupby("행정동명")['Y'].transform("mean"), inplace=True)


In [ ]:

#이 외 invalid 다 탈락
data=data.dropna(axis=0)

In [ ]:
#index다시 만들어주기
data.reset_index()
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
#-성공여부
r=[0,730,999999]#일단 2년을 성공기준
g=[0,1]
data['성공여부']=pd.cut(data['영업기간'],bins=r,labels=g)
data['성공여부']=data['성공여부'].fillna(0)#확인사살
#data=data.drop('영업기간',axis=1)
data['성공여부']

In [ ]:
#-창업월
data["인허가일자"] = pd.to_datetime(data["인허가일자"])
data['창업월'] = data['인허가일자'].dt.month
data['창업월']

In [ ]:
#-폐업월
data["폐업일자"] = pd.to_datetime(data["폐업일자"])
data['폐업월'] = data['폐업일자'].dt.month
#data=data.drop('인허가일자',axis=1)
data['폐업월']

In [ ]:
data['년도_123']=data['년도']-1960
r=[0,40,50,80]
g=[1,2,3]
data['년도_123']=pd.cut(data['년도_123'],bins=r,labels=g)
#data=data.drop('년도',axis=1)
data['년도_123']

In [ ]:
#gu mapping
gu_mapping={"강남구":1, "강동구":2, "강북구":3, '강서구':4,'관악구':5,'광진구':6,'구로구':7,'금천구':8,'노원구':9,'도봉구':10,'동대문':11,'동작구':12,'마포구':13,'서대문':14,'서초구':15,'성동구':16,'성북구':17,'송파구':18,'양천구':19,'영등포':20,'용산구':21,'은평구':22,'종로구':23,'중구':24,'중랑구':25}
data['구_1_25']=data['자치구명'].map(gu_mapping)

#data=data.drop('자치구명',axis=1)
data['구_1_25']

In [ ]:
#호선 추출하기
data['sub_1_9']=data['호선'].str[:2]
data['sub_1_9']

In [ ]:
import plotly.express as px
is_franchise=data['사업장명'].str.contains("이디야|스타벅스|투썸|달콤커피|커피베이|할리스|탐앤탐스|폴바셋|커피나무|파스쿠찌|파스쿠치|빽다방|빈스빈스|엔제리너스|엔젤리너스|커피명가|카페베네|커피스미스|드롭탑|커피마마|더착한커피|더카페|커피에반하다|매머드커피|주커피|토프레소|카페보니또|전광수커피|그라찌에|띠아모|커피빈|더벤티|스무디킹")
#사업장명이 없는 건 프랜차이즈 아닌걸로하자
is_super_franchise=data['사업장명'].str.contains("이디야|스타벅스|투썸")
data["프랜차이즈여부_02"]=0
is_franchise=is_franchise.fillna(0)
is_super_franchise=is_super_franchise.fillna(0)
is_franchise=is_franchise.astype(int)
is_super_franchise=is_super_franchise.astype(int)
for i in range(0,40766):
  try:
    data["프랜차이즈여부_02"][i]=is_franchise[i]+is_super_franchise[i]
  except KeyError:
    continue
data["프랜차이즈여부_02"]

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
def bar_charty(feature):
  Success=data[data['성공여부']==1][feature].value_counts()
  Fail=data[data['성공여부']==0][feature].value_counts()
  df=pd.DataFrame([Success,Fail])
  df.index=['성공',"실패"]
  df.plot(kind='bar',stacked=True, figsize=(10,5))

In [ ]:
bar_charty('자치구명')

In [ ]:
bar_charty('영업상태명')

In [ ]:
bar_charty('규모분기')

In [ ]:
bar_charty('업소위치분기')#

In [ ]:
bar_charty('창업월')

In [ ]:
 bar_charty('폐업월') 

In [ ]:
bar_charty('년도_123')

In [ ]:
bar_charty('프랜차이즈여부_02')

In [ ]:
bar_charty('sub_1_9')

In [ ]:
#루미하네 캡쳐본 보냄
facet=sns.FacetGrid(data, hue='당월_매출_금액', aspect=4)
facet.map(sns.kdeplot, 'd_code',shade=True)
facet.set(xlim=(data['d_code'].min(),data['d_code'].max()))
facet.add_legend() 

import matplotlib.pyplot as plt
plt.show()
#1영업, 2폐업


In [ ]:
#루미한테 캡쳐본 보냄
facet=sns.FacetGrid(data, hue='당월_매출_금액', aspect=4)
facet.map(sns.kdeplot, 'distance',shade=True)
facet.set(xlim=(data['distance'].min(),3))
facet.add_legend() 

import matplotlib.pyplot as plt
plt.show()
#1영업, 2폐업


In [ ]:
facet=sns.FacetGrid(data, hue='점포수', aspect=4)
facet.map(sns.kdeplot, 'distance',shade=True)
facet.set(xlim=(data['distance'].min(),4))
facet.add_legend() 

import matplotlib.pyplot as plt
plt.show()
#1영업, 2폐업

In [ ]:
data.head()

In [ ]:
#이건...노답
facet=sns.FacetGrid(data, hue='프랜차이즈여부_02', aspect=4)
facet.map(sns.kdeplot, '당월_매출_금액',shade=True)
facet.set(xlim=(data['당월_매출_금액'].min(),data['당월_매출_금액'].max()))
facet.add_legend() 

import matplotlib.pyplot as plt
plt.show()
#1영업, 2폐업프랜차이즈여부_02

In [ ]:
facet=sns.FacetGrid(data, hue='당월_매출_금액', aspect=4)
facet.map(sns.kdeplot, '프랜차이즈여부_02',shade=True)
facet.set(xlim=(0.75,1.25),ylim=(0,0.5))
facet.add_legend() 

import matplotlib.pyplot as plt
plt.show()
#1영업, 2폐업프랜차이즈여부_02